<a href="https://colab.research.google.com/github/jiwoong2/deeplearning/blob/main/Utill_%EC%9E%90%EC%97%B0%EC%96%B4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

In [ ]:
text = 'You say goodbye and I say hello.'

In [ ]:
# 희소행렬. 말뭉치에 등장하는 순서대로 단어를 원-핫 인코딩 한다.

def preprocess(text):
    text = text.lower() # 소문자 변환
    text = text.replace('.', ' .') # 특정 단어 대체
    words = text.split(' ') # 띄어쓰기로 문장을 분리. 리스트로 반환

    word_to_id = {}
    id_to_word = {}
    for word in words:
        if word not in word_to_id:
            new_id = len(word_to_id)
            word_to_id[word] = new_id
            id_to_word[new_id] = word

    corpus = np.array([word_to_id[w] for w in words])

    return corpus, word_to_id, id_to_word

In [ ]:
corpus, word_to_id, id_to_word = preprocess(text)

In [ ]:
# 동시 발생 행렬. 분포 가셀에 따라 주변에 등장하는 단어를 벡터에 반영한다.

def create_co_matrix(corpus, vocab_size, window_size=1): # vocab_size 는 단어의 개수이다.

    corpus_size = len(corpus)
    co_matrix = np.zeros((vocab_size, vocab_size), dtype=np.int32)

    for idx, word_id in enumerate(corpus): # 인덱스와 원소를 묶어서 반환.
        for i in range(1, window_size + 1):
            left_idx = idx - i
            right_idx = idx + i

            if left_idx >= 0:
                left_word_id = corpus[left_idx]
                co_matrix[word_id, left_word_id] += 1

            if right_idx < corpus_size:
                right_word_id = corpus[right_idx]
                co_matrix[word_id, right_word_id] += 1

    return co_matrix

In [ ]:
C = create_co_matrix(corpus, 7)

# 코사인 유사도

In [ ]:
def cos_similarity(x, y, eps = 1e-8):

    nx = x / (np.sqrt(np.sum(x ** 2)) + eps) # x를 normalize
    ny = y / (np.sqrt(np.sum(y ** 2)) + eps) # y를 normalize

    return np.dot(nx, ny)

In [ ]:
# you와 i의 유사도를 비교.

c0 = C[word_to_id['you']]
c1 = C[word_to_id['i']]
c0, c1

In [ ]:
# 결과로 변환된 벡터에 단어의 본질적인 의미가 어느정도 녹아들었다는것을 알 수 있다.

cos_similarity(c0, c1)

# 코사인 유사도 높은순으로 출력하기

In [ ]:
def most_similar(query, word_to_id, id_to_word, word_matrix, top=5):
    '''유사 단어 검색

    :param query: 쿼리(텍스트)
    :param word_to_id: 단어에서 단어 ID로 변환하는 딕셔너리
    :param id_to_word: 단어 ID에서 단어로 변환하는 딕셔너리
    :param word_matrix: 단어 벡터를 정리한 행렬. 각 행에 해당 단어 벡터가 저장되어 있다고 가정한다.
    :param top: 상위 몇 개까지 출력할 지 지정
    '''
    if query not in word_to_id:
        print('%s(을)를 찾을 수 없습니다.' % query)
        return

    print('\n[query] ' + query)
    query_id = word_to_id[query]
    query_vec = word_matrix[query_id]

    # 코사인 유사도 계산
    vocab_size = len(id_to_word)

    similarity = np.zeros(vocab_size)
    for i in range(vocab_size):
        similarity[i] = cos_similarity(word_matrix[i], query_vec)

    # 코사인 유사도를 기준으로 내림차순으로 출력
    count = 0
    for i in (-1 * similarity).argsort(): # argsort() 오름차순으로 인덱스를 반환한다.
        if id_to_word[i] == query:
            continue
        print(' %s: %s' % (id_to_word[i], similarity[i]))

        count += 1
        if count >= top:
            return

In [ ]:
# 말 뭉치에서 you와 코사인 유사도가 가장 가까운 단어를 찾는다.
# 하지만 말 뭉치가 너무 짧기 때문에 그리 정확하지 않다.

most_similar('you', word_to_id, id_to_word, C, top=5)

# PPMI

In [ ]:
def ppmi(C, verbose=False, eps = 1e-8):
    '''PPMI(점별 상호정보량) 생성

    :param C: 동시발생 행렬
    :param verbose: 진행 상황을 출력할지 여부
    :return:
    '''
    M = np.zeros_like(C, dtype=np.float32)
    N = np.sum(C)
    S = np.sum(C, axis=0)
    total = C.shape[0] * C.shape[1]
    cnt = 0

    for i in range(C.shape[0]):
        for j in range(C.shape[1]):
            pmi = np.log2(C[i, j] * N / (S[j]*S[i]) + eps)
            M[i, j] = max(0, pmi)

            if verbose:
                cnt += 1
                if cnt % (total//100 + 1) == 0:
                    print('%.1f%% 완료' % (100*cnt/total))
    return M

In [ ]:
W = ppmi(C)

In [ ]:
np.set_printoptions(precision=3) # 유효 자릿수 세 자리.
print(W)

# 차원 축소(PCA)

In [ ]:
text = 'You say goodbye and I say hello.'

In [ ]:
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(id_to_word)
C = create_co_matrix(corpus, vocab_size, window_size=1)
W = ppmi(C)

In [ ]:
# SVD(singular value decomposition)
U, S, V = np.linalg.svd(W)

In [ ]:
np.set_printoptions(precision = 3)
print(C[0])
print(W[0])
print(U[0])
print(U)

In [ ]:
import matplotlib.pyplot as plt

for word, word_id in word_to_id.items():
    plt.annotate(word, (U[word_id, 0], U[word_id, 1])) # 좌표에 주석달기.

plt.scatter(U[:,0], U[:,1], alpha = 0.5) # 2 차원으로 차원을 축소.
plt.show()